In [1]:
import numpy as np 
import pandas as pd 
import nltk

# load the data
df=pd.read_csv('train.csv')
dft=pd.read_csv('test.csv')
dfs = pd.read_csv('sample_submission.csv')
df=pd.concat([df, dft], axis=0)
df = df.replace(np.nan,'0')

df['text'] = df[['text', 'keyword']].agg(''.join, axis=1)

# remove punctuation
punc=[
    '?','#','!','@','#','$','%','^','&','*',
    '(',')','_','+','=','-','<','>',',','.',
     ]
     
# for p in punc:
#       df['text'] = df.apply(lambda x:x['text'].replace(p,'')  , axis=1)
for p in punc:
    df=df.replace(p,'')
# tokkening

df['text'] = df.apply(lambda row:nltk.word_tokenize(row['text'] ) , axis=1)






                                    

    




                               


In [2]:
from nltk.stem.snowball import SnowballStemmer

# Use English stemmer.
stemmer = SnowballStemmer("english")
# stemming
df['text'] = df['text'].apply(lambda x: [stemmer.stem(y) for y in x])


In [3]:
# vectorizer
vocab= []
for sentence in df['text']:
    for word in sentence:
         if word not in vocab:
            vocab.append(word)
          

In [4]:
#Index dictionary to assign an index to each word in vocabulary
index_word = {}
i = 0
for word in vocab:
    index_word[word] = i 
    i += 1

In [5]:
from collections import defaultdict
#number of words in the vocab
len_vector = len(vocab)
def bag_of_words(sent):
    count_dict = defaultdict(int)
    vec = np.zeros(len_vector)
    for item in sent:
        count_dict[item] += 1
    for key,item in count_dict.items():
        vec[index_word[key]] = item
    return vec  

In [6]:
df['text'] = df.apply(lambda row:bag_of_words(row['text'] ) , axis=1)



In [7]:
df['keyword'] = df.apply(lambda row:bag_of_words(row['keyword'] ) , axis=1)


In [8]:
df=df.drop(columns='location')


In [9]:

# df['text'] = df.apply(lambda row:np.concatenate((np.array(row['text']), np.array(row['keyword'])))  , axis=1)



In [10]:
x=df.drop(columns=['target','keyword'])
df_1=x.iloc[0:7613,:]
df_2=x.iloc[7613:,:]

In [14]:
import scipy 
from scipy import sparse
from scipy.sparse import csr_matrix

arr_1=df_1['text'].to_numpy()
sarr_1=np.stack( arr_1, axis=0 )
a=sparse.csr_matrix(sarr_1)

arr_2=df_2['text'].to_numpy()
sarr_2=np.stack( arr_2, axis=0 )
b=sparse.csr_matrix(sarr_2)


In [16]:
from sklearn import linear_model
clf = linear_model.RidgeClassifier()
tar=pd.read_csv('train.csv')
clf.fit(a, tar["target"])



RidgeClassifier()

In [17]:
dfs['target']= clf.predict(b)
dfs.set_index('id', inplace=True)

dfs.to_csv('prediction_results_2.csv')